
# POC 激活验证

In [7]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP
from utils_gnn import unpadding, padding




# x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_0.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)
del params

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
del model_detail

import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

# get a image
image = None
label = None
for i, (x_in, y_in) in enumerate(dataloader):
    image = x_in
    model(image)
    label = y_in
    break
del trainset, dataloader

None
{'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}
None
{'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}
None
None
None
1 in 32 out: torch.Size([1, 1, 28, 28])
conv1 weight size: torch.Size([32, 1, 5, 5])
32 in 32 out torch.Size([1, 32, 28, 28])
32 in 16 out: torch.Size([1, 32, 28, 28])
16 in 8 out: torch.Size([1, 16, 14, 14])


In [8]:
def my_conv(data, data_size, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # print(weight.size())
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size[0], kernel_size[1])
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    print("kernel_size, stride, padding:")
    print(kernel_size, stride, padding)
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, 
                    stride=stride, padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    print("conv input size:", data.size())
    x = operator(data.to("cuda"))
    print("conv outpu size:", x.size())
    convd_size = x.size()
    return x, convd_size


def maxpool(kernel_size, stride, padding):
    pass  

def message_func(edges):
    # print("this is message function.")
    # print("message fucntion ends")
    # print("edges src ft size:", edges.src['ft'].size())
    return {'m': edges.src['ft']}
    # return {}


def reduce(nodes):
    # print("this is reduce function")

    pre_data = nodes.mailbox['m']
    # print("mailbox data size:", pre_data.size())
    ft = torch.zeros((1, 1, 28, 28)).to("cuda")
    # print("nodes data size in reduce:", ft.size())
    # print("nodes ft size:", nodes.data['ft'].size())
    if pre_data.all() == 0:
        return {'h': torch.sum(pre_data, dim=1)}
    for i in range(len(pre_data)):
        # else do conv operation
        kernel_size, stride, padding = nodes.data['params'][i]

        res_ft = my_conv(pre_data[i][0], None, nodes.data['x'][i], nodes.data['bias'][i], kernel_size, stride, padding)
        # print("res_ft in reduce func:", res_ft)
        ft += res_ft
    # print("size before and after:", nodes.mailbox['m'].size(), ft.size())
    # print("reduce function ends")
    return {'h': ft}


def initiate_node_feature(graph, image):
    ft = None 
    mask = graph.ndata['tag'] == 0
    out_channels = int(sum(mask))
    ft = torch.zeros((len(graph.nodes()), 1, 1, 28, 28))
    convd_size = torch.zeros((len(graph.nodes()), 2))
    for i in range(out_channels):
        kernel_size, stride, padding = graph.ndata['params'][i]
        res_ft, ret_size = my_conv(image, None, graph.ndata['x'][i], graph.ndata['bias'][i],
                kernel_size, stride, padding)
        print("ret_size: ", ret_size)
        _, _, r, c = ret_size
        ft[i] = res_ft
        convd_size[i] = torch.tensor([int(r), int(c)])
    graph.ndata['ft'] = ft.to("cuda")
    graph.ndata['convd_size'] = convd_size.to("cuda")


def conv_message_passing(graph):
    # get node id first, then get subgraph
    def nodes_feature_smaller_than_one(nodes):
        return nodes.data['tag'] <= 1
    
    nodes_idx = graph.filter_nodes(nodes_feature_smaller_than_one)
    # print("in conv mp, nodes idx:", nodes_idx)
    subg = dgl.node_subgraph(graph, nodes_idx)
    def conv_reduce_func():
        pass
    subg.update_all(dgl.function.copy_src('ft', 'm'))
    ft = subg.ndata['ft']

def cnn_cal(graph, image):
    initiate_node_feature(graph, image)
    conv_message_passing(graph)
    # graph.update_all(message_func=message_func, reduce_func=reduce_func)
    #ft = graph.ndata['ft'][0]
    #return ft

import json
# get cnn results
pred, params = model(x_in)
print(params.keys())
with open("./intermediate_data/params.json", "w") as f:
    json.dump(params, f)
# get gnn transmission results
with torch.no_grad():
    res = cnn_cal(g, image)


1 in 32 out: torch.Size([1, 1, 28, 28])
conv1 weight size: torch.Size([32, 1, 5, 5])
32 in 32 out torch.Size([1, 32, 28, 28])
32 in 16 out: torch.Size([1, 32, 28, 28])
16 in 8 out: torch.Size([1, 16, 14, 14])
dict_keys(['conv1', 'conv2', 'conv3', 'conv4'])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
conv outpu size: torch.Size([1, 1, 28, 28])
ret_size:  torch.Size([1, 1, 28, 28])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
conv outpu size: torch.Size([1, 1, 28, 28])
ret_size:  torch.Size([1, 1, 28, 28])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
conv outpu size: torch.Size([1, 1, 28, 28])
ret_size:  torch.Size([1,

TypeError: update_all() missing 1 required positional argument: 'reduce_func'

In [ ]:
# def conv_reduce_func(nodes):
#     pass
# def reduce_func(nodes):
#     print("this is reduce function")

#     # input_mask = nodes.data['tag'] == 0
#     # conv_mask = nodes.data['tag'] == 1
#     # concat_mask = nodes.data['tag'] == 2
#     # fc_mask = nodes.data['tag'] == 3

#     # input_feat = nodes.data['x'][input_mask]
#     # print(input_feat)
#     pre_data = nodes.mailbox['m']
#     print("mailbox data size:", pre_data.size())
#     print("nodes data size in reduce:", nodes.data['x'].size())
#     ft = torch.zeros((28, 28)).to("cuda")
#     if pre_data.all() == 0:
#         return {'h': ft}
#     print("nodes ft size:", nodes.data['ft'].size())
#     for i in range(len(pre_data)):
#         # else do conv operation
#         kernel_size, stride, padding = nodes.data['params'][i]
#         res_ft = conv(pre_data[i][0], nodes.data['x'][i], nodes.data['bias'][i],
#                 kernel_size, stride, padding)
#         print("res_ft in reduce func:", res_ft)
#         # check if max pooling
#         ft += res_ft
#     print("reduce function ends")
#     return {'h': ft}

In [ ]:
print(g.ndata['tag'])
# get node id
def nodes_with_feature_smaller_two(nodes):
    return nodes.data['tag'] <= 2
nodes_idx = g.filter_nodes(nodes_with_feature_smaller_two)
subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
print(subg.nodes())
ft32 = None

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3],
       device='cuda:0')
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88],
       device='cuda:0')


In [15]:
def do_conv():
    pass

def my_reduce(nodes):
    m = nodes.mailbox['m']
    actual_size = nodes.mailbox['size']
    print("mailbox data size:", m.size(), actual_size.size())
    print(nodes)
    mshape = m.size()
    sshape = actual_size.size()
    n, f, d = actual_size.size()
    ret_size = torch.zeros((n, d)).to("cuda")
    # reduce nodes batch N
    for out_idx in range(mshape[0]):
        if m[out_idx].all() == 0:
            continue
        # begin the process for each node
        received_feats = m[out_idx]
        received_sizes = actual_size[out_idx]
        channels = received_feats.size()[0]
        ft = torch.zeros((1, 1, 28, 28)).to("cuda")
        print("channels: %d " % channels)

        # prepare conv operator
        kernel_size, stride, pad = nodes.data['params'][i]
        conv_operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, stride=stride, padding=pad)
        # get and set my_conv weight & bias
        weight = nodes.data['x'][i]
        w = unpadding(weight, kernel_size[0], kernel_size[1])
        w = w.unsqueeze(0).unsqueeze(0)
        bias = nodes.data['bias'][i]
        b = unpadding(bias, 1, 1)[0]
        print("weight size:", conv_operator.weight.size())
        conv_operator.weight.data = w
        print("weight size:", conv_operator.weight.size())
        conv_operator.bias.data = b
        row, col = -1, -1
        # for in_channel, data_size, size_idx in zip(received_feats, actual_size[out_idx], ret_size[0]):
        for inner_idx in range(len(received_feats)):
            in_channel = received_feats[inner_idx]
            data_size = received_sizes[inner_idx]
            # get actual size data
            row, col = data_size
            row, col = int(row), int(col)
            print("types: ", type(row))
            print("row: ", row)
            # resize 
            data = in_channel.resize_((28, 28))
            data = unpadding(data, row, col)
            data = data.resize_((1, 1, row, col))
        
            # do conv
            print("input size:", data.size())
            data = conv_operator(data.to("cuda"))
            print("finished conv, res size:", data.size())

            # do relu
            relu_operator = torch.nn.functional.relu
            data = relu_operator(data)

            # do max_pool
            pooling = nodes.data['pooling'][out_idx]
            if pooling.all() != 0:
                # do max_pooling
                kernel_size, stride, pad, dilation, ceil_mode = pooling
                max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                    padding=pad, dilation=dilation, ceil_mode=ceil_mode)
                
                data = max_pooling_operator(data)
            _, _, row, col = data.size()
            data = data.resize_((row, col))
            print("maxPooling data size: ", data.size(), type(data))
            data = padding(data, 28, 28)
            ft += data
        convd_size = torch.tensor([row, col])
        ret_size[out_idx] = convd_size.to("cuda")



    # return size is [n, 1, 1, 28, 28], reduced from [n, m, 1, 1, 28, 28]
    return {'h': torch.sum(m, dim=1), 'ret_size': ret_size}

def my_message(edges):
    return {'m': edges.src['ft'], 'ret_size': edges.src['convd_size']}

with torch.no_grad():
    subg.update_all(my_message, my_reduce)
    subg.ndata['convd_size'] = subg.ndata['ret_size']

def print_zeros_in_feat(key):
    print(key + ": ")
    zeros = []
    nonzero = []
    for id in subg.nodes():
        id = int(id)
        feat = subg.nodes[id].data[key]
        if feat.all() == 0:
            zeros.append(id)
        else:
            nonzero.append(id)
    print("zeros - ", zeros)
    print("nonzeros - ", nonzero)
print_zeros_in_feat('ft')
print_zeros_in_feat('h')
subg.ndata['ft'] += subg.ndata['h']
cur_ft32 = subg.ndata['ft'][32]
if ft32 is not None and torch.eq(cur_ft32, ft32).all():
    ft32 = cur_ft32
    print("ft32 equals!")
else:
    ft32 = cur_ft32
    print("ft32 not equeal!")


mailbox data size: torch.Size([1, 8, 1, 1, 28, 28]) torch.Size([1, 8, 2])
mailbox data size: torch.Size([8, 16, 1, 1, 28, 28]) torch.Size([8, 16, 2])
mailbox data size: torch.Size([48, 32, 1, 1, 28, 28]) torch.Size([48, 32, 2])
channels: 32 
weight size: torch.Size([1, 1, 3, 3])
weight size: torch.Size([1, 1, 3, 3])
types:  <class 'int'>
row:  28
input size: torch.Size([1, 1, 28, 28])
finished conv, res size: torch.Size([1, 1, 28, 28])
maxPooling data size:  torch.Size([28, 28]) <class 'torch.Tensor'>
types:  <class 'int'>
row:  28
input size: torch.Size([1, 1, 28, 28])
finished conv, res size: torch.Size([1, 1, 28, 28])
maxPooling data size:  torch.Size([28, 28]) <class 'torch.Tensor'>
types:  <class 'int'>
row:  28
input size: torch.Size([1, 1, 28, 28])
finished conv, res size: torch.Size([1, 1, 28, 28])
maxPooling data size:  torch.Size([28, 28]) <class 'torch.Tensor'>
types:  <class 'int'>
row:  28
input size: torch.Size([1, 1, 28, 28])
finished conv, res size: torch.Size([1, 1, 28

RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (3 x 3). Kernel size can't be greater than actual input size